In [1]:
import numpy as np
import matplotlib.animation
import matplotlib.pyplot as plt
import random
from IPython.display import HTML
from matplotlib import animation
import timeit

In [2]:
def energy(sigma, J, H):
    ''' Calculate lattice Energy from Hamiltonian
        E = -J * sum(S(i)*S(j);i^j) - H * sum(S(j);j)
     Parameters
     ----------
     sigma : Full lattice that includes boundaries
     H : External field
     J : Interaction 
    '''
    return (-J * np.sum(sigma[:-1,:] * sigma[1:,:]) + -J * np.sum(sigma[:,:-1] * sigma[:,1:]) + -H * np.sum(sigma))

In [3]:
def setBoundary(sigma):
    sigma[0,:]  = sigma[-1,:]
    sigma[-1,:] = sigma[-2,:]
    sigma[:,0]  = sigma[1,:]
    sigma[:,-1] = sigma[2,:]
    return

In [4]:
def mcflip_sublat(sigma,a,b, J, H):
    """Update on sublattices defined by and randomly flip with probability
    set by the relative Gibbs weight."""
    
    # localH acts on the bulk sublattice which is defined by the slice
    # sigma[1+a:-1:2, 1+b:-1:2]
    Lx, Ly = sigma.shape
    localH = H* sigma[1+a:-1:2, 1+b:-1:2] + J * sigma[1+a:-1:2, 1+b:-1:2]*(
            sigma[2+a:   :2, 1+b: -1:2] + # to the right
            sigma[  a: -2:2, 1+b: -1:2] + # to the left
            sigma[1+a: -1:2, 2+b:   :2] + # up
            sigma[1+a: -1:2,   b: -2:2]   # down 
            )
    
    #metropolis probability:
    p = np.exp(-2 * localH)
    sigma[1+a:-1:2, 1+b:-1:2] *= -1 * (2*(np.random.rand(Lx//2-1,Ly//2-1) < p) - 1)          
    return

In [5]:
#flips the spins and sets the periodic boundaries
def mcflip(sigma, J, H):
    setBoundary(sigma)
    mcflip_sublat(sigma,0,0, J, H)
    setBoundary(sigma)  
    mcflip_sublat(sigma,1,0, J, H)
    setBoundary(sigma)  
    mcflip_sublat(sigma,0,1, J, H)
    setBoundary(sigma)  
    mcflip_sublat(sigma,1,1, J, H)
    setBoundary(sigma)  
    return

In [6]:
def acf(x, length=20):
    ''' Auto Correlation Function'''
    return np.array([1]+[np.corrcoef(x[:-i], x[i:])[0,1]  \
    for i in range(1, length)])

In [7]:
def tp_corr(sigma,n,length=50):#two point correlation function 
    L=len(sigma)
    C=[]
    for i in range(1,length):
        Xi=[]
        Xj=[]
        for j in range(n):
            x=np.random.randint(0,L)
            y=np.random.randint(0,L)
            Xi+=[sigma[x][y]]
            Xj+=[sigma[x][(y+i)%L]]
        Xi=np.array(Xi)
        Xj=np.array(Xj)
        C+=[(np.mean(Xi*Xj)-np.mean(Xi)*np.mean(Xj))/np.var(Xi)]     
    return C

In [8]:
def corr_len(auto_corr):
    for i, ac in enumerate(auto_corr):
        if (np.abs(ac - (1/np.e)) <= 0.005) and (np.abs(ac - (1/np.e)) < np.abs(auto_corr[(i+1)%len(auto_corr)] - (1/np.e))):
            return 

In [9]:
def magnetization(sigma):
    return np.mean(sigma[1:-1,1:-1]) # exclude boundary spins

In [10]:
def runSample(nSteps = 10000, subSample = 10):
    """Run nSteps Monte Carlo sweeps on the 2D Ising model.
       Does not initialize the sample so can be called sequentially without
       reinitialization if desired.
   
       Returns:
           histMag -- array of magnetizations at every subSample timesteps
           histEn  -- array of energies at every subSample timesteps
   """
   
   # create lists to grow
    histMag = []
    histEn = []
        
    for t in range(nSteps):
        mcflip(sigma, J, H)
                 
        if t % subSample == 0:
            histMag.append(magnetization(sigma))
            histEn.append(energy(sigma, J, H))
    # return as arrays
    return (np.array(histMag), np.array(histEn))

In [11]:
def run_sample_ksi(nSteps,subSample):
    """this function takes samples to calculate the positional correlation length 
       using the functions tp_corr
    """
    histksi = []
    for t in range(nSteps):
        mcflip(sigma, J, H)    
        if t % subSample == 0:
            ksi=corr_len(tp_corr(sigma,50,length=20))
            if ksi is not None: 
                histksi.append(ksi)
            else:
                histksi.append(0)
    # return as arrays
    return np.array(histksi)

In [11]:
def Generate(sigma,J,H):
    '''
    this function is the automatic data creator from our simulation
    it takes 6000 samples at first, dumps the first 2000 and then goes on with the method 
    explained in the report to sample efficiently and with good quality
    '''
    Mag0, En0= runSample(6000,1) 
    ac = acf(Mag0[2000:],length=900)
    step = corr_len(ac)
    Mag, En = Mag0[2000::step], En0[2000::step]
    if step is None:                              #when step is less than 1 we set step=1
        step=1              
    if 1000//step < 100:                          #when step is big we get more data 
        Mag0, En0=runSample((500*step-1000),step)
        Mag = np.append(Mag,Mag0)
        En = np.append(En,En0)
     
   
    return(Mag, En, step)

In [12]:
Jcrit = np.log(1+np.sqrt(2.))/2 # J in Critical temperatur

In [13]:
lx=100
ly=100
J=Jcrit
H=0
M=[]
sigma=np.random.choice([1,-1],size=(lx,ly))
for i in range (6000) :
    mcflip(sigma,J,H)
    M.append(magnetization(sigma))

برای اجرای انیمیشن، همه کامنت ها را بردارید

In [18]:
#fig = plt.figure(figsize=(8,8)) 
#plt.title('Ising model for lattice size = 100 * 100 in J = J_critical') 
#im = plt.imshow(sigma, vmin = -1, vmax = 1)
#def animate(i):
    #mcflip(sigma,Jcrit, H)
    #im.set_data(sigma)
    #return
  
#ani = matplotlib.animation.FuncAnimation(fig, animate, frames=1000, interval=1000/30)

#from IPython.display import HTML
#HTML(ani.to_jshtml())

In [15]:
lx=100
ly=100
J=Jcrit * 1.5
H=0
M=[]
sigma=np.random.choice([1,-1],size=(lx,ly))
for i in range (6000) :
    mcflip(sigma,J,H)
    M.append(magnetization(sigma))

In [17]:
#fig = plt.figure(figsize=(8,8))
#plt.title('Ising model for lattice size = 100 * 100 in J = 1.5 * J_critical') 
#im = plt.imshow(sigma, vmin = -1, vmax = 1)
#def animate(i):
    #mcflip(sigma,J, H)
    #im.set_data(sigma)
    #return
  
#ani = matplotlib.animation.FuncAnimation(fig, animate, frames=1000, interval=1000/30)

#from IPython.display import HTML
#HTML(ani.to_jshtml())

In [19]:
lx=100
ly=100
J=Jcrit * 0.5
H=0
M=[]
sigma=np.random.choice([1,-1],size=(lx,ly))
for i in range (6000) :
    mcflip(sigma,J,H)
    M.append(magnetization(sigma))

In [20]:
#fig = plt.figure(figsize=(8,8))
#plt.title('Ising model for lattice size = 100 * 100 in J = 0.5 * J_critical') 
#im = plt.imshow(sigma, vmin = -1, vmax = 1)
#def animate(i):
    
    #mcflip(sigma,J, H)
    #im.set_data(sigma)
    #return
  
#ani = matplotlib.animation.FuncAnimation(fig, animate, frames=1000, interval=1000/30)

#from IPython.display import HTML
#HTML(ani.to_jshtml())

In [21]:
lx=100
ly=100
J=Jcrit * 0.2
H=0
M=[]
sigma=np.random.choice([1,-1],size=(lx,ly))
for i in range (6000) :
    mcflip(sigma,J,H)
    M.append(magnetization(sigma))

In [22]:
#fig = plt.figure(figsize=(8,8))
#plt.title('Ising model for lattice size = 100 * 100 in J = 0.2 * J_critical') 
#im = plt.imshow(sigma, vmin = -1, vmax = 1)
#def animate(i):
    #mcflip(sigma,J, H)
    #im.set_data(sigma)
    #return
  
#ani = matplotlib.animation.FuncAnimation(fig, animate, frames=1000, interval=1000/30)

#from IPython.display import HTML
#HTML(ani.to_jshtml())